Для подсчета эффективности работы SMM-команды используют метрику индекс вовлеченности. Индекс рассчитывается как отношение количества подписчиков группы к сумме комментариев, лайков и репостов всех записей за определенный период. Чем этот показатель меньше, тем более активно подписчики группы реагируют на публикуемый в ней контент.

Выясним какой индекс вовлеченности у  https://vk.com/popularmechanics

Для вычислений будем использовать следующий упрощенный алгоритм:

1. Выгрузим 100 последних постов из каждой группы

2. Посчитаем сумму комментариев, лайков и репостов этих публикаций для каждой группы

3. Получим количество подписчиков каждой группы (это число возвращает метод groups.getMembers)

4. Вычислим индекс вовлеченности для каждой группы

#### выгрузим 100 последних постов для https://vk.com/popularmechanics

In [1]:
from pprint import pprint

In [2]:
import requests
from yaml import load
from pprint import pprint
with open('config.yaml', 'r') as f:
    config = load(f)
token = config['access_token']
url = 'https://api.vk.com/method/wall.get'
params_pm = {
    'domain': 'popularmechanics',
    'filter': 'owner',
    'count': 100,
    'offset': 0,
    'access_token': token,
    'v': 5.73
}
data_pm = requests.get(url, params = params_pm)


#### Нужная нам статистика находится в полях 'comments', 'likes' и 'reposts'. Соберем итоговую статистику для каждого поста в словарь stats. 

In [3]:
Started = True
stats_pm = {}
for record in data_pm.json()['response']['items']:
    if Started:
        Started = False
    
    else:
        if 'attachments' in record:
            if 'link' in record['attachments'][0]:
                title = record['attachments'][0]['link']['title']
            elif 'photo' in record['attachments'][0]:
                title = record['attachments'][0]['photo']['text']
        elif 'copy_history' in record:
            if 'link' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['link']['title']
            elif 'poll' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['poll']['question']
        stats_pm[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]
pprint(stats_pm)


{'': [0, 61, 5, 1541232816],
 '10 гигантских беспозвоночных': [6, 80, 10, 1541168707],
 '10 настоящих американских грузовиков': [9, 129, 21, 1541325680],
 '10 фактов о «Кока-коле»': [19, 64, 7, 1541499196],
 '14 миллиардов лет в одном видео: эволюция Вселенной': [23,
                                                         159,
                                                         39,
                                                         1541170924],
 '5 автомобилей-амфибий, которые вы можете купить прямо сейчас': [7,
                                                                  47,
                                                                  10,
                                                                  1541568607],
 '5 книг о танках': [2, 40, 0, 1541166164],
 '8 самых интересных событий фестиваля премии «Просветитель 2018»': [1,
                                                                     26,
                                                              

#### Получим топ-10 постов по сумме лайков, комментариев и репостов

In [4]:
for title_pm, data_pm in sorted( stats_pm.items(), key = lambda x: -(x[1][0]+x[1][1]+x[1][2]) )[:10]:
    print(title_pm, data_pm)


Спорткар против гоночного дрона: видео [23, 373, 39, 1541585703]
14 миллиардов лет в одном видео: эволюция Вселенной [23, 159, 39, 1541170924]
Первое видео из космоса в разрешении 8K [42, 154, 19, 1541515594]
Нейроимплантаты, помогающие ходить парализованным людям [12, 132, 23, 1541484221]
«Небесный дворец»: новая космическая станция Китая [46, 102, 13, 1541579875]
10 настоящих американских грузовиков [9, 129, 21, 1541325680]
Женская грудь отпугивает мужчин от попытки реанимации [72, 72, 7, 1541577605]
Как пожар ядерным взрывом тушили [19, 107, 11, 1541416641]
Первый смартфон с по-настоящему складным экраном [22, 96, 16, 1541158603]
Реплика Me.262 в воздухе [15, 108, 5, 1541608401]


#### Переведем время публикации постов из UNIX-формата в дату:

In [5]:
from datetime import datetime, timedelta
for title_pm, line_pm in stats_pm.items():
    line_pm.append(datetime.fromtimestamp(line_pm[3]).strftime('%Y-%m-%d'))
    print(line_pm)

[1, 14, 3, 1541750407, '2018-11-09']
[7, 21, 2, 1541749207, '2018-11-09']
[1, 41, 4, 1541748004, '2018-11-09']
[7, 24, 3, 1541746992, '2018-11-09']
[4, 30, 3, 1541745833, '2018-11-09']
[2, 9, 1, 1541744405, '2018-11-09']
[5, 15, 0, 1541743210, '2018-11-09']
[2, 37, 3, 1541742005, '2018-11-09']
[2, 48, 5, 1541740805, '2018-11-09']
[43, 40, 1, 1541739605, '2018-11-09']
[16, 70, 2, 1541696108, '2018-11-08']
[1, 33, 6, 1541690592, '2018-11-08']
[5, 42, 5, 1541689198, '2018-11-08']
[19, 63, 14, 1541686256, '2018-11-08']
[32, 37, 6, 1541684519, '2018-11-08']
[13, 33, 3, 1541674185, '2018-11-08']
[2, 44, 7, 1541667606, '2018-11-08']
[8, 35, 5, 1541665806, '2018-11-08']
[6, 37, 5, 1541664006, '2018-11-08']
[0, 42, 7, 1541662208, '2018-11-08']
[1, 21, 1, 1541661000, '2018-11-08']
[32, 38, 2, 1541660405, '2018-11-08']
[17, 55, 5, 1541658606, '2018-11-08']
[8, 35, 1, 1541656805, '2018-11-08']
[8, 37, 3, 1541655005, '2018-11-08']
[18, 58, 8, 1541653205, '2018-11-08']
[37, 52, 2, 1541612178, '2018-

#### Получаем вчерашнюю дату:

In [6]:
yesterday = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
print(yesterday)

2018-11-08


#### Выводим обновленный рейтинг лучших постов с фильтром на вчерашнюю дату:

In [7]:
for title_pm, line_pm in sorted( stats_pm.items(), key = lambda x: -(x[1][0]+x[1][1]+x[1][2]) ):
    if line_pm[4] == yesterday:
        print(title_pm, line_pm)

Можно ли уничтожить автомобиль мойкой? [19, 63, 14, 1541686256, '2018-11-08']
Су-57 сняли на видео с рампы Ан-12 [16, 70, 2, 1541696108, '2018-11-08']
Это безопасно: наноботы, которые просверлят вам глаз [18, 58, 8, 1541653205, '2018-11-08']
Спасет ли телефонный справочник от пуль: бумажная броня [17, 55, 5, 1541658606, '2018-11-08']
Первый электрокар Audi: слезая с нефтяной иглы [32, 37, 6, 1541684519, '2018-11-08']
Мощный лазерный сигнал мог бы сообщить инопланетянам о Земле [32, 38, 2, 1541660405, '2018-11-08']
Гаджет, который точно скажет, какими аллергенами вы дышите [2, 44, 7, 1541667606, '2018-11-08']
Эндоскопия: как выглядит человек изнутри [5, 42, 5, 1541689198, '2018-11-08']
Samsung показала прототип смартфона со сгибающимся экраном [13, 33, 3, 1541674185, '2018-11-08']
Анимированная история альпийских ледников [0, 42, 7, 1541662208, '2018-11-08']
Первый банкомат с искусственным интеллектом [8, 35, 5, 1541665806, '2018-11-08']
В Китае теперь могут идентифицировать человека по

In [8]:
cur_sum_pm=0
tot_sum_pm=0
for title_pm, line_pm in sorted( stats_pm.items(), key = lambda x: -(x[1][0]+x[1][1]+x[1][2]) ):
    if line_pm[4] == yesterday:
        cur_sum_pm=line_pm[0]+line_pm[1]+line_pm[2]
        tot_sum_pm+=cur_sum_pm
print('сумма лайков и репостов публикаций для Popular Mechanics за вчерашний день составляет:', tot_sum_pm)

сумма лайков и репостов публикаций для Popular Mechanics за вчерашний день составляет: 946


In [9]:
import requests
from yaml import load
from pprint import pprint
with open('config.yaml', 'r') as f:
    config = load(f)
token = config['access_token']
url_getmembers = 'https://api.vk.com/method/groups.getMembers'

params_pm = {
    'group_id': 'popularmechanics',
    'v': 5.73,
    'access_token': token
}
r_pm = requests.get(url_getmembers, params = params_pm)
data_pm = r_pm.json()
#print('количество подпищиков popularmechanics составляет:',(data_pm))
print('количество подписчиков в группе popularmechanics составляет:',data_pm['response']['count'])

количество подписчиков в группе popularmechanics составляет: 989352


In [10]:
import requests
from yaml import load
from pprint import pprint
with open('config.yaml', 'r') as f:
    config = load(f)
token = config['access_token']
url_getmembers = 'https://api.vk.com/method/groups.getMembers'

params_hb = {
    'group_id': 'habr',
    'v': 5.73,
    'access_token': token
}
r_hb = requests.get(url_getmembers, params = params_hb)
data_hb = r_hb.json()
#print('количество подпищиков popularmechanics составляет:',(data_pm))
print('количество подписчиков в группе habr составляет:',data_hb['response']['count'])

количество подписчиков в группе habr составляет: 781905


In [11]:
print('индекс вовлеченности для Popular Mechanics за вчерашний день составляет:', data_pm['response']['count'] / tot_sum_pm)

индекс вовлеченности для Popular Mechanics за вчерашний день составляет: 1045.8266384778012
